In [1]:
import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize
import glob

In [2]:
#Read in CSV data files

airbnb = pd.read_csv('zip_clean.csv',index_col=0)
bydz_availcount = pd.read_csv('bydz_availcount.csv',header=None)
bydz_availcount.columns=['date_zip','avail_count']
bydzrt_roomcount = pd.read_csv('bydzrt_roomcount.csv',header=None)
bydzrt_roomcount.columns=['date_zip','room_type','date','bydzrt_count']
prr_df = pd.read_csv('prr_df.csv',index_col=0)
mlp_df = pd.read_csv('mlp_df.csv',index_col=0)
air_zillow = pd.read_csv('airzillow_df.csv', index_col=0)

In [3]:
data = pd.read_csv('Gaz_zcta_national.txt', sep="\t")
data.columns = ["zcta", "pop10", "hu10", "aland","awater","aland_sqmi","awater_sqmi","lat","long"]
data.head()


,zcta,pop10,hu10,aland,awater,aland_sqmi,awater_sqmi,lat,long
0,601,18570,7744,166659789,799296,64.348,0.309,18.180555,-66.749961
1,602,41520,18073,79288158,4446273,30.613,1.717,18.362268,-67.176130
2,603,54689,25653,81880442,183425,31.614,0.071,18.455183,-67.119887
3,606,6615,2877,109580061,12487,42.309,0.005,18.158345,-66.932911
4,610,29016,12618,93021467,4172001,35.916,1.611,18.290955,-67.125868


In [4]:
airbnb15 = airbnb.loc[airbnb['year'] == 2015]
airbnb15.to_csv('airbnb15.csv')
airbnb16 = airbnb.loc[airbnb['year'] == 2016]
airbnb16.to_csv('airbnb16.csv')
airbnb17 = airbnb.loc[airbnb['year'] == 2017]
airbnb17.to_csv('airbnb17.csv')

In [5]:
airbnb17_count = airbnb17[['id','date_zip','zipcode','month']].groupby(['date_zip','zipcode','month'], as_index=False).agg('count')
airbnb17_count = pd.merge(airbnb17_count, data[['zcta','pop10','lat','long']],how='left', left_on='zipcode', right_on='zcta')
airbnb17_count = airbnb17_count[['date_zip','zipcode','month','id','pop10','lat','long']]
airbnb17_count = airbnb17_count.rename(columns={'id': 'airbnb_count'})
airbnb17_count['norm_count'] = airbnb17_count['airbnb_count'].div(airbnb17_count['pop10'])
nullpop = airbnb17_count.loc[airbnb17_count['pop10']==0][['date_zip','zipcode','month','airbnb_count','pop10','lat','long']]
nullpop
airbnb17_count = airbnb17_count.loc[airbnb17_count['pop10']!=0]
airbnb17_count
airbnb17_count.to_csv('airbnb17_count.csv')

# print(type(airbnb17_count['zipcode']))
# print(type(data['zcta']))

In [6]:
airbnb17_countavg = airbnb17_count[['zipcode','airbnb_count','pop10','norm_count']].groupby(['zipcode'],as_index=False).agg('mean')
airbnb17_countavg.to_csv('airbnb17_countavg.csv')

In [7]:
sortnorm_17 = airbnb17_countavg.sort_values(by='norm_count',ascending=False)
topnorm_17 = sortnorm_17.loc[sortnorm_17['norm_count']>=0.01]
sortnorm_17.head()

,zipcode,airbnb_count,pop10,norm_count
15,10018,179.6,5229.0,0.034347
10,10012,594.7,24090.0,0.024687
32,10036,598.3,24711.0,0.024212
106,11222,710.8,36934.0,0.019245
12,10014,574.3,31959.0,0.017970


In [8]:
mlp_avg = mlp_df[['median_list_price','year','zip']].groupby(['zip','year'], as_index=False).agg('mean')
mlp_avg17 = mlp_avg[['median_list_price','year','zip']].loc[mlp_avg['year']==2017]
mlp_avg17.head()

,median_list_price,year,zip
1,3.030000e+06,2017,10001
5,1.588250e+06,2017,10002
8,2.546750e+06,2017,10003
11,1.150167e+06,2017,10004
20,1.439875e+06,2017,10005


In [9]:
airbnb17.head()

,accommodates,availability_30,availability_365,availability_60,availability_90,bathrooms,bedrooms,beds,calculated_host_listings_count,date,...,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,year,zipcode
678010,2.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,1.0,2017-01-01,...,8.0,8.0,7.0,9.0,10.0,80.0,9.0,Entire home/apt,2017,11233
678012,3.0,12.0,288.0,29.0,47.0,1.0,1.0,1.0,1.0,2017-01-01,...,10.0,10.0,9.0,9.0,9.0,96.0,10.0,Entire home/apt,2017,11103
678013,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2017-01-01,...,10.0,8.0,9.0,8.0,6.0,90.0,10.0,Private room,2017,11221
678015,3.0,4.0,154.0,34.0,64.0,1.0,1.0,1.0,1.0,2017-01-01,...,10.0,10.0,10.0,10.0,10.0,100.0,9.0,Entire home/apt,2017,10018
678016,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2017-01-01,...,9.0,10.0,10.0,10.0,10.0,92.0,9.0,Entire home/apt,2017,10022


In [10]:
airbnb17_price = airbnb17[['zipcode','price']].groupby(['zipcode'], as_index=False).agg('mean')

In [11]:
topzip17 = pd.merge(topnorm_17,airbnb17_price,how='left',on='zipcode')
# topzip17 = topzip17.dropna(axis=0,how='any')
topzip17 = topzip17[['zipcode','norm_count','price']]
topzip17.to_csv('topzip17.csv')
topzip17

,zipcode,norm_count,price
0,10018,0.034347,229.126949
1,10012,0.024687,224.851354
2,10036,0.024212,208.302691
3,11222,0.019245,120.881401
4,10014,0.017970,227.079401
5,11238,0.017866,130.694239
6,11211,0.017666,146.601759
7,10019,0.016930,200.083219
8,11216,0.016538,107.068129
9,10001,0.016055,213.537485


In [12]:
topnormprice_17 = topzip17.sort_values(by='price',ascending=False)
topnormprice_17

,zipcode,norm_count,price
0,10018,0.034347,229.126949
4,10014,0.017970,227.079401
1,10012,0.024687,224.851354
20,10013,0.011996,223.137526
14,10011,0.013283,217.563940
9,10001,0.016055,213.537485
10,10006,0.015277,209.171739
2,10036,0.024212,208.302691
12,10003,0.014672,200.157908
7,10019,0.016930,200.083219


In [13]:
sortnorm_17

norm_17 = pd.merge(sortnorm_17,airbnb17_price,how='left',on='zipcode')
# topzip17 = topzip17.dropna(axis=0,how='any')
norm_17 = norm_17[['zipcode','norm_count','price']]
norm_17.to_csv('norm17.csv')
norm_17

,zipcode,norm_count,price
0,10018,0.034347,229.126949
1,10012,0.024687,224.851354
2,10036,0.024212,208.302691
3,11222,0.019245,120.881401
4,10014,0.017970,227.079401
5,11238,0.017866,130.694239
6,11211,0.017666,146.601759
7,10019,0.016930,200.083219
8,11216,0.016538,107.068129
9,10001,0.016055,213.537485


In [14]:
air_zillow.head()

,date_zip,active_count,pr_ratio,median_list_price,month,year,zip,date
0,201501_10001,297,24.23,NaN,1,2015,10001,2015-01
1,201501_10002,902,19.59,1290000.0,1,2015,10002,2015-01
2,201501_10003,718,24.52,NaN,1,2015,10003,2015-01
3,201501_10004,44,23.50,3550000.0,1,2015,10004,2015-01
4,201501_10005,111,23.86,1300000.0,1,2015,10005,2015-01


In [15]:
az_df = air_zillow.dropna()
az_df.head()

,date_zip,active_count,pr_ratio,median_list_price,month,year,zip,date
1,201501_10002,902,19.59,1290000.0,1,2015,10002,2015-01
3,201501_10004,44,23.50,3550000.0,1,2015,10004,2015-01
4,201501_10005,111,23.86,1300000.0,1,2015,10005,2015-01
6,201501_10007,31,28.11,870000.0,1,2015,10007,2015-01
9,201501_10011,667,24.14,1855000.0,1,2015,10011,2015-01


In [16]:
df = pd.merge(az_df, data[['zcta','pop10','lat','long']],how='left', left_on='zip', right_on='zcta')
df.head()

,date_zip,active_count,pr_ratio,median_list_price,month,year,zip,date,zcta,pop10,lat,long
0,201501_10002,902,19.59,1290000.0,1,2015,10002,2015-01,10002,81410,40.715762,-73.986258
1,201501_10004,44,23.50,3550000.0,1,2015,10004,2015-01,10004,3089,40.688630,-74.018244
2,201501_10005,111,23.86,1300000.0,1,2015,10005,2015-01,10005,7135,40.705974,-74.008768
3,201501_10007,31,28.11,870000.0,1,2015,10007,2015-01,10007,6988,40.713864,-74.007806
4,201501_10011,667,24.14,1855000.0,1,2015,10011,2015-01,10011,50984,40.742002,-74.000594


In [17]:
df.to_csv('linear_reg_df.csv')